## LIMPEZA DOS DADOS

In [ ]:
import pandas as pd
import tensorflow as tf
import seaborn as sb
import numpy as np
import matplotlib.pyplot as plt

# Carregando e fazendo visualizações iniciais
df = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")
test_y = pd.read_csv("gender_submission.csv")
df.set_index('PassengerId')
test_y.set_index('PassengerId')
df.head()

In [ ]:
print(df.nunique()) # Verificando a quantidade de valores nulos


Aqui se pode verificar que não há discrepâncias claras de quantidade de valores categoricos parar as colunas categóricas

- Verificação dos valores nulos:

In [ ]:
print("TRAIN:")
print(df.isna().sum()) # Verificando se há valores nulos
print("TEST:")
print(test.isna().sum())


Aqui vemos uma grande quantidade de valores nulos em Cabin, o que indica que remover a coluna seria uma boa escolha. Além disso, precisamos encontrar uma maneira de tratar as idades que estão faltando.

- Verificação das correlações:

In [ ]:
correl = df.drop('PassengerId', axis=1)
sb.heatmap(correl.corr())

A partir do heatmap não foi possível encontrar boas correlações e nem mesmo uma boa coluna a partir da qual inferir os valores de idade que estão faltando no dataset original

- Verificação dos valores categóricos:

In [ ]:
print(df.dtypes)

As colunas Ticket, Name e Cabin podem ser removidas por serem inerentemente irrelevantes. A coluna Embarked também não parece ser relevante para a análise final.

In [ ]:
cols = ['Name', 'Cabin', 'Ticket', 'Embarked']
df.drop(cols, axis=1, inplace=True)
test.drop(cols, axis=1, inplace=True)
df.head()

Agora, basta substituir os valores dos sexos para 0: male e 1: female. Por termos baixa cardinalidade, é possível usar um One-Hot Encoder

In [ ]:
from sklearn.preprocessing import OneHotEncoder

In [ ]:
hot_enc = OneHotEncoder()
df_hot = pd.get_dummies(df['Sex'])
test_hot = pd.get_dummies(test['Sex'])

df = df.drop(['Sex'], axis='columns')
test = test.drop(['Sex'], axis='columns')

df = pd.concat([df, df_hot], axis='columns')
test = pd.concat([test, test_hot], axis='columns')
df.head()

- Análise de Outliers para o treino

In [ ]:
# Criando dataframe para verificar outliers
out = df.drop(['female', 'male', 'PassengerId', 'Survived'], axis=1)

fig, axs = plt.subplots(ncols=out.columns.size, figsize=(20,5))
for i in range(out.columns.size):
    sb.boxplot(data=out.iloc[:, i], ax=axs[i])
    axs[i].set_title(out.columns[i])
plt.show()

Aqui podemos ver que existem dois valores de outliers para a idade, alguns para SibSp e Parch e muitos para Fare. Nesse caso, não tendo descartado passenger anteriormente, talvez esse seja um bom motivo. Embora passageiros que tenham pago mais possam estar acomodadas em locais específicos que podem ter sido mais ou menos afetados, essa informação também deve estar representada em Pclass, no entanto, o mapa de correlação visto anteriormente nos diz que Fare tem uma das melhores relações com Survived, portanto talvez não seja razoável removê-la. À princípio, irei removê-la, mas também é possível realizar teste com a coluna.

In [ ]:
df.drop('Fare', axis=1, inplace=True)
test.drop('Fare', axis=1, inplace=True)

Por cautela, podemos verificar a contagem de passageiros para cada valor de SibSp e Parch a partir dos valores que estão fora dos desvios padrão:

In [ ]:
print("Parch")
for i in range(1, df['Parch'].max()):
    print(f'{i}:', len(df[df['Parch'] == i]))

print("SibSp")
for i in range(2, df['SibSp'].max()):
    print(f'{i}:', len(df[df['SibSp'] == i]))
df.shape


 Aqui podemos ver que é razoável remover os outliers a partir de 3 em Parch e a partir de 3 em SibSp

In [ ]:
df = df[(df['Parch'] <= 3) & (df['SibSp'] <= 3)]
out = df.drop(['female', 'male', 'PassengerId', 'Survived'], axis=1)

fig, axs = plt.subplots(ncols=out.columns.size, figsize=(20,5))
for i in range(out.columns.size):
    sb.boxplot(data=out.iloc[:, i], ax=axs[i])
    axs[i].set_title(out.columns[i])
plt.show()

Assim removemos a maior parte dos outliers sem remover uma quantidade extremamente grande de linhas da tabela.

- Imputation

Por fim, precisamos encontrar uma forma de ajustar os valores perdidos de idade. À princípio, podemos imputar os valores com base na média. É interessante notar que o dataset de teste também possui idades não preenchidas. Nesse caso, no entanto, é melhor remover as linhas, haja visto que precisamos testar com dados completos.

In [ ]:
from sklearn.impute import SimpleImputer

In [ ]:
imputer = SimpleImputer()
df_imputed = pd.DataFrame(imputer.fit_transform(df))

# Devolvendo os nomes das colunas:
df_imputed.columns = df.columns
df_imputed.set_index('PassengerId')
print(df_imputed.head())
print(df_imputed.isna().sum())
print("Média idade: ", df['Age'].mean())
df = df_imputed

In [ ]:
test = test[~test['Age'].isna()]
test_y = test_y[test_y.PassengerId.isin(test.PassengerId)] # Removendo os valores da resposta
print(test_y.shape)
print(test.shape)

In [ ]:
# Criando dataframe para verificar outliers
out = df.drop(['female', 'male', 'PassengerId'], axis=1)

fig, axs = plt.subplots(ncols=out.columns.size, figsize=(20,5))
for i in range(out.columns.size):
    sb.boxplot(data=out.iloc[:, i], ax=axs[i])
    axs[i].set_title(out.columns[i])
plt.show()

In [ ]:
df['PassengerId'] = df['PassengerId'].astype(int) # Corrigindo o tipo do Id
df = df.set_index('PassengerId')
df

Aqui temos a nova tabela pronta para ser treinada pelos modelos de Machine Learning.

In [ ]:
test = test.set_index('PassengerId')
test_y = test_y.set_index('PassengerId')
print(test)

# TREINANDO MODELOS

Agora já temos datasets de treino e de teste e podemos começar a comparar modelos.

- #### RANDOM FOREST

In [ ]:
from sklearn.metrics import f1_score, accuracy_score, mean_absolute_error, precision_score, recall_score
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

Primeiramente, precisamos separar os datasets em treino e validação

In [ ]:
X = df.drop('Survived', axis=1)
y = df.Survived

train_x, val_x, train_y, val_y = train_test_split(X, y, random_state=1)

E também façamos o Min-Max Scaling da variável Age

In [ ]:
max = train_x['Age'].max()
min = train_x['Age'].min()

for i in range(train_x['Age'].size):
    train_x['Age'].iloc[[i]] = (train_x['Age'].iloc[[i]] - min) / (max - min)

print(train_x['Age'].max())
print(train_x['Age'].min())

Vamos verificar o balanceamento do dataset

In [210]:
print(f'Porcentagem de 0s: {round((train_y[train_y == 0].count() / train_y.shape[0]) * 100)}%')
print(f'Porcentagem de 1s: {round((train_y[train_y == 1].count() / train_y.shape[0]) * 100)}%')

Porcentagem de 0s: 61%
Porcentagem de 1s: 39%


Embora o balanceamento não esteja perfeito, deve ser suficiente para uma boa predição dos modelos

Agora implementando o modelo de Random Forest

In [ ]:
rf = RandomForestClassifier(random_state=1, n_estimators=1000, criterion='log_loss')
rf.fit(train_x, train_y)

# Observando o erro no treinamento
predict = rf.predict(val_x)
rf_f1 = f1_score(predict, val_y)
print("Validation f1-score for Random Forest: {}".format(rf_f1))

Aqui temos um erro aproximadamente razoável, para o modelo treinado com o subset de treino e validação, vamos ver como ele lida com as previsões.

No entanto, como estamos lidando com uma saída booleana, podemos verificar outras métricas de assertividade do modelo.

Nesse caso, Accuracy, Precision e Recall:

In [ ]:
# Verificação de métricas
def metricas(modelo):
    valores_corretos = np.array(test_y.squeeze())
    predict_final = modelo.predict(test)

    acc = accuracy_score(predict_final, valores_corretos)
    recall = recall_score(predict_final, valores_corretos)
    precision = precision_score(predict_final, valores_corretos)
    f1score = f1_score(predict_final, valores_corretos)

    print(f'Accuracy: {round(acc * 100)}%')
    print(f'Precision: {round(precision * 100)}%')
    print(f'Recall: {round(recall * 100)}%')
    print(f'f1-score: {round(f1score * 100)}%')

In [ ]:
metricas(rf)

Aqui verificamos boa acurácia, mediano f1-score e ótimo recall, embora tenhamos precisão razoável. Isso significa que o modelo não tem muitos false negatives, ou seja, é um modelo conservador.

- Teste de hiperparâmetros

Agora vamos fazer testes com diferentes parâmetros para as árvores para verificar se podemos encontrar algum modelo mais adequado.

Primeiramento, uma floresta com 200 árvores e critério de erro "gini"

In [ ]:
rf_2 = RandomForestClassifier(n_estimators=200, criterion="gini", verbose=0, random_state=1)

In [ ]:
rf_2.fit(train_x, train_y)

predict = rf_2.predict(val_x)
rf_f1 = f1_score(predict, val_y)
print("f1-score para o conjunto de validação:", rf_f1)

In [ ]:
metricas(rf_2)

Aqui não vemos muitas mudanças na capacidade de predição do modelo

Agora, uma floresta com 300 árvores usando entropy.

In [ ]:
rf_3 = RandomForestClassifier(n_estimators=300, criterion="entropy", verbose=0, random_state=1)

In [ ]:
rf_3.fit(train_x, train_y)

predict = rf_3.predict(val_x)
rf_f1 = f1_score(predict, val_y)
print("f-1 score para o conjunto de validação:", rf_f1)

In [ ]:
metricas(rf_3)

Novamente, não se observa grandes mudanças no modelo

- ### LOGISTIC REGRESSOR

In [195]:
from sklearn.linear_model import LogisticRegression

In [221]:
lr = LogisticRegression(penalty='l2', class_weight=None)

lr.fit(train_x, train_y)
predict = lr.predict(val_x)
print(f'f1-score for Logistic Regression training: {f1_score(predict, val_y)}')
print(f'Accuracy for Logistic Regression training: {accuracy_score(predict, val_y)}')

predict[:30]

f1-score for Logistic Regression training: 0.0
Accuracy for Logistic Regression training: 0.5915492957746479


array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

Note que o modelo está prevendo sempre o valor 0, o que mantém o f1 score em 0, por não ter recall, mas garante uma acurácia razoável. Isso pode ser um indicativo de que os datasets de treino estão mal balanceados.

Vamos fazer um undersampling para tentar melhorar os resultados de previsão do modelo:

In [242]:
print(f'Number of 1s: {train_y[train_y == 1].count()}')
print(f'Number of 0s: {train_y[train_y == 0].count()}')

train_y.drop(index=0)

Number of 1s: 251
Number of 0s: 387


KeyError: '[0] not found in axis'

In [241]:
for i in range(50):
    if (train_y.iloc[[i]] == 0).values[0]:
        new_train_y = train_y.drop(index = train_y.iloc[[i]].name)

KeyError: '[0] not found in axis'